### Fluvial sanity / comparison overview

Anchors:
- `scripts.smoke_test`
- `analog_image_generator.geologic_generators.generate_fluvial`

Use this notebook to demonstrate the features listed below. Fill in the upcoming sections (presets, generator runs, stacked packages, metrics, reporting, and debugging hooks) before the professor review.


# anchor-fluvial-sanity
## Fluvial Preview Sanity Check
Code anchor: `analog_image_generator.preview.generate_preview`

This notebook renders a lightweight preview for the fluvial pipeline so we can visually sanity check
each implementation increment without waiting for full UX/reporting wiring. It pairs with the Task Master
implementation loop described in Section 8 of `docs/CODEX_RUNBOOK.md`.

#### Demo checklist

- [ ] Load smoke test assets for all styles
- [ ] Side-by-side comparison plots (meandering/braided/anasto)
- [ ] Stacked vs single overlay
- [ ] Seed toggle/debug cell
- [ ] Link to `outputs/smoke_report/` artifacts


### Workflow Notes
- Run `scripts/render_preview.py --env fluvial` to reproduce the assets saved here.
- Update Task Master before/after each run so previews map back to task IDs.
- When generators mature, swap placeholder metadata for actual masks + QC flags.

### TODO: populate fluvial sanity demo cells

- Add preview generation once `analog_image_generator.preview.generate_preview` supports metadata masks.
- Display smoke-test artifacts from `outputs/smoke_report/`.
- Include seed toggle + Task Master snapshot cells when ready.
